In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler

%matplotlib inline                               


Download From https://drive.google.com/file/d/1lcLgDvpj_yRCKsjH-eG6Km46VY3bakys/view?usp=sharing

In [ ]:
train_df = pd.read_pickle("./train_5.pkl")
valid_df = pd.read_pickle("./valid_5.pkl")
test_df = pd.read_pickle("./test_5.pkl")

In [ ]:
## Family DataSet
X_train = train_df.drop('label',axis=1).values.astype(np.float64)
X_valid = valid_df.drop('label',axis=1).values.astype(np.float64)
X_test = test_df.drop('label',axis=1).values.astype(np.float64)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

realation_dict = {'fs':0, 'fd':1, 'ms':2, 'md':3, 'un':4}
y_train = train_df['label'].map(realation_dict).values.astype(np.int64)
y_valid = valid_df['label'].map(realation_dict).values.astype(np.int64)
y_test = test_df['label'].map(realation_dict).values.astype(np.int64)

train_dataset = TensorDataset(torch.from_numpy(X_train).type(torch.FloatTensor), torch.from_numpy(y_train))
valid_dataset = TensorDataset(torch.from_numpy(X_valid).type(torch.FloatTensor), torch.from_numpy(y_valid))
test_dataset = TensorDataset(torch.from_numpy(X_test).type(torch.FloatTensor), torch.from_numpy(y_test))

loaders = {}
loaders['train'] = DataLoader(train_dataset, batch_size=200, shuffle=True)
loaders['valid'] = DataLoader(valid_dataset, batch_size=200)
loaders['test'] = DataLoader(test_dataset, batch_size=200)

## Family Model Development

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 64)
        self.output = nn.Linear(64, 5)
        
        self.dropout = nn.Dropout(p=0.5)
        self.logSoftMax = nn.LogSoftmax(dim=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.logSoftMax(self.output(x))
        return x

model = Net()
use_cuda = torch.cuda.is_available()
if use_cuda:
    model.cuda()

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# initialize tracker for minimum validation loss
valid_loss_min = np.Inf 

for epoch in range(400):
    # initialize variables to monitor training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        optimizer.zero_grad()
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        ## find the loss and update the model parameters accordingly
        pred = model(data)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        ## record the average training loss, using something like
        train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))

    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['valid']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        ## update the average validation loss
        pred = model(data)
        loss = criterion(pred, target)
        valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
    if(epoch % 10 == 0):
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

    ## TODO: save the model if validation loss has decreased
    if valid_loss < valid_loss_min:
        if(epoch % 10 == 0):
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
        torch.save(model.state_dict(), "model5_checkpoint.cpt")
        valid_loss_min = valid_loss

In [ ]:
model.load_state_dict(torch.load('model5_checkpoint.cpt'))
# monitor test loss and accuracy
test_loss = 0.
correct = 0.
total = 0.

model.eval()
for batch_idx, (data, target) in enumerate(loaders['test']):
    # move to GPU
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update average test loss 
    test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
    # convert output probabilities to predicted class
    pred = output.data.max(1, keepdim=True)[1]
    # compare predictions to true label
    correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
    total += data.size(0)

print('Test Loss: {:.6f}\n'.format(test_loss))

print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
    100. * correct / total, correct, total))